In [65]:
#%cd ..

In [1]:
import requests
import json
import re
import numpy as np
import pandas as pd

from parser.org import TEST_URL

# (Down)load the JSON to File

In [4]:
# headers = requests.utils.default_headers()
# headers.update({'User-Agent': 'Mozilla/5.0',})
# raw = requests.get(TEST_URL, headers=headers).text

# with open('test_org.json', 'w') as fp:
#     fp.write(raw)

In [5]:
with open('test_org.json', 'r') as json_file:
    data = json.load(json_file)

# URL's Entity's UUID

In [57]:
uuid = data['properties']['identifier']['uuid']
this_uuid = uuid
uuid

'ce3d7079-97a0-e664-66cf-add27867cee1'

# Utility Functions

In [13]:
def dict_generator(indict, pre=None):
    pre = pre[:] if pre else []
    if isinstance(indict, dict):
        for key, value in indict.items():
            if isinstance(value, dict):
                for d in dict_generator(value, pre + [key]):
                    yield d
            elif isinstance(value, list) or isinstance(value, tuple):
                for v in value:
                    for d in dict_generator(v, pre + [key]):
                        yield d
            else:
                yield pre + [key, value]
    else:
        yield pre + [indict]

In [48]:
def get_only_values(d, ngram=1, ngram_join=' '):
    acc = []
    for g in dict_generator(d):
        if 'recommended_search' not in g:
            acc.append(ngram_join.join([str(x) for x in g[-ngram:]]))
    return acc

# Most Common JSON Leaves (Lookback 2 --> 3grams)

In [72]:
from collections import Counter
Counter(get_only_values(data, ngram=3, ngram_join=' <-- ')).most_common(20)

[('identifier <-- entity_def_id <-- organization', 30),
 ('identifier <-- image_id <-- v1421524538/gasqwdjyarqy2vv84guh.png', 27),
 ('identifier <-- entity_def_id <-- job', 25),
 ('person_identifier <-- entity_def_id <-- person', 25),
 ('identifier <-- uuid <-- ce3d7079-97a0-e664-66cf-add27867cee1', 22),
 ('identifier <-- value <-- Manhattan Venture Partners', 22),
 ('identifier <-- permalink <-- manhattan-venture-partners', 22),
 ('identifier <-- entity_def_id <-- builtwith_technology', 10),
 ('product_identifier <-- entity_def_id <-- siftery_product', 10),
 ('siftery_product_list <-- status <-- using', 10),
 ('identifier <-- entity_def_id <-- investment', 10),
 ('organization_identifier <-- entity_def_id <-- organization', 10),
 ('funding_round_identifier <-- entity_def_id <-- funding_round', 10),
 ('identifier <-- entity_def_id <-- hub', 10),
 ('current_advisors_image_list <-- job_type <-- advisor', 7),
 ('funding_round_money_raised <-- currency <-- USD', 6),
 ('started_on <-- preci

# Find UUID Out-Links
Finds all the referenced uuids (that are not the uuid of this entity) with their
respective counts and all referenced types in the JSON file

In [73]:
def find_uuid_outlinks(d):
    uuids = {}
    counts = {}
    for v in get_only_values(d, ngram=3, ngram_join='**'):
        [ref, lit, uuid] = v.split('**')
        if lit != 'uuid' or uuid == this_uuid:
            continue
        
        if not uuid in uuids:
            uuids[uuid] = [ref]
            counts[uuid] = 1
        else:
            uuids[uuid].append(ref)
            counts[uuid] += 1

    return uuids, counts

In [79]:
u, c = find_uuid_outlinks(data)

acc = []
for uuid in u:
    acc.append((c[uuid], uuid + ' : ' + str(u[uuid])))
    
sorted(acc, reverse=True)

[(3,
  "603d3264-7a74-8e61-accb-78a9ec863356 : ['founder_identifiers', 'person_identifier', 'person_identifier']"),
 (3,
  "4b850877-9c2e-aeb3-6d47-e4ea1286d8c1 : ['founder_identifiers', 'person_identifier', 'person_identifier']"),
 (2,
  "f99dac7d-460c-455e-8f67-89544f9aaa16 : ['identifier', 'siftery_product_list']"),
 (2, "e89d7d05-74bc-3514-f791-d127f5c274db : ['identifier', 'identifier']"),
 (2, "d710114b-64f2-61c8-70ce-103f9c86d89c : ['identifier', 'identifier']"),
 (2, "c4d7e661-4328-ee4f-4970-0cc426685028 : ['identifier', 'identifier']"),
 (2,
  "b42fc8fb-dc6f-4081-9639-10b31186c0e7 : ['identifier', 'siftery_product_list']"),
 (2,
  "ad470638-5fa1-46b4-9631-14bea999022a : ['identifier', 'siftery_product_list']"),
 (2,
  "a16e610e-fab1-4e58-b87e-1baed30aeb1c : ['identifier', 'siftery_product_list']"),
 (2,
  "97fa0976-de84-183a-0f01-8c1776e67796 : ['person_identifier', 'person_identifier']"),
 (2,
  "7444f056-c329-42c3-903f-10fbe6cd4f77 : ['identifier', 'siftery_product_list']"),